# 自分で実装

In [56]:
import numpy as np
import scipy as sp

リカッチ代数方程式
$$
PA+A^{\top}P-PBR^{-1}B^{\top}P+Q=0
$$
の正定解$P$を求める．  

In [57]:
A = np.array([
    [1.1, 2],
    [-0.3, -1],
])
B = np.array([
    [1, 2],
    [0.847, 3],
])
Q = np.diag([10.0, 10.0])
R = np.diag([1., 1.])

P_solver = sp.linalg.solve_continuous_are(A, B, Q, R)
P_solver  # ソルバを使って求めた解

array([[ 3.34568216, -1.07265801],
       [-1.07265801,  1.30235279]])

# リカッチの非線形行列微分方程式を解いて求める
[教科書](#1)のp.160にある方法．力技．こちらは簡単．  

In [58]:
def solve_care(A, B, Q, R):
    P = np.zeros_like(A)
    P_ = P.copy()
    
    invR = np.linalg.inv(R)
    
    while True:
        P += (P @ A + A.T @ P - P @ B @ invR @ B.T @ P + Q) * 0.001
        if np.abs(P - P_).max() < 1e-12:
            break
        P_ = P.copy()
    
    return P


P = solve_care(A, B, Q, R)
P

array([[ 3.34568216, -1.07265801],
       [-1.07265801,  1.30235279]])

scipyの結果と比較

In [59]:
P - P_solver

array([[-1.81811899e-10,  7.89479593e-11],
       [ 7.89479593e-11, -3.42814666e-11]])

ほぼ同じ．

****
## 有本・ポッターの方法
[教科書](#1)のp.160にある方法．  
詳しくは[julia](../julia_src/simple.ipynb)を見てください．  

In [69]:
def arimoto_potter(A, B, Q, R):
    """有本・ポッターの方法"""
    n = len(A)
    H = np.block([
        [A, -B @ np.linalg.inv(R) @ B.T],
        [-Q, -A.T],
    ])  # ハミルトン行列
    
    L, W = np.linalg.eig(H)  # 固有値と固有ベクトル
    U, V = [], []
    for i, l in enumerate(L):
        if np.real(l) < 0:  # 固有値が負の最適極の場合
            U.append(W[:n, i:i+1])
            V.append(W[n:, i:i+1])
    U = np.concatenate(U, axis=1)
    V = np.concatenate(V, axis=1)
    
    return V @ np.linalg.inv(U)

P_ap = arimoto_potter(A, B, Q, R)
P_ap

array([[ 3.34568216, -1.07265801],
       [-1.07265801,  1.30235279]])

In [70]:
P_ap - P_solver

array([[-3.55271368e-15, -4.44089210e-16],
       [ 2.22044605e-15,  4.44089210e-16]])